# 此文件用于筛选fpocket的输出结果

In [100]:
import os
import copy

# 函数定义

In [122]:
def is_dibind(file, base=0) -> bool:
    # 用于判断pocket文件中是否同时有蛋白质残基和RNA残基
    # return bool
    with open(file, 'r') as f:
        cont = 1
        for line in f:
            # print(line)
            if line[:4] != 'ATOM': continue
            if cont == 1:
                residue_name_length = len(f.readline()[17:20].strip())
            residue_name = line[17:20].strip()
            cont += 1
            if len(residue_name) == residue_name_length:
                continue
            else:
                return True
    return False

def read_druggability(pockets_info):
    score_dict = dict()
    with open(pockets_info, 'r') as f:
        pockets = f.read().strip().split('\n\n')
        for i, p in enumerate(pockets):
            i += 1
            score_dict[i] = float(p.split('\n')[2].split('\t')[2])
    return score_dict

def fpocket_filter(fpocket_out_folder, low_line=0.5):
    # 此函数用于过滤pocket，利用di_bind函数和分数
    pocket_folder = os.path.join(fpocket_out_folder, 'pockets')
    pockets_info_file = os.path.join(fpocket_out_folder, fpocket_out_folder.split('/')[-1][:-4]+'_info.txt')
    # pockets_file = [os.path.join(pocket_folder, file) for file in os.listdir(pocket_folder) if file[-3:]=='pdb']
    pockets_score = read_druggability(pockets_info_file)
    results = list()
    for i in pockets_score.keys():
        pocket_file_path = os.path.join(pocket_folder,f'pocket{i}_atm.pdb')
        if pockets_score[i] > low_line and is_dibind(pocket_file_path):
            results.append(i)
    return results

def rewrite_out_file(pdb_out_file:str, pockets:list):
    # 此函数用于将不在pockets的pocket删除，并将剩余的pocket重新编号
    with open(pdb_out_file, 'r') as f:
        line_list = f.read().strip().split('\n')
        # out_line = copy.deepcopy(line_list)
        for i, line in enumerate(line_list):
            # print(i)
            line_list[i] = line + '\n'
            line += '\n'
            if line[:6] == 'HETATM' and line[17:20] == 'STP':
                if int(line[22:26]) not in pockets:
                    line_list[i] = ''
                else:
                    line_list[i] = line[:22] + (' '*(4-len(str(pockets.index(int(line[22:26]))+1))) + str(pockets.index(int(line[22:26]))+1)) + line[26:]

    with open(pdb_out_file.replace('.pdb', '_new.pdb'), 'w') as f:
        f.writelines(line_list)
        

# main函数

In [123]:
if __name__ == "__main__":
    # fpocket输出文件夹，例："E:\tmp\fpocket\6v5b_out"
    fpocket_out_folder = r'/mnt/e/tmp/fpocket/6v5b_out'
    pdb_out_file = os.path.join(fpocket_out_folder, fpocket_out_folder.split('/')[-1]+'.pdb')
    pockets = fpocket_filter(fpocket_out_folder, low_line=0.1)
    # print(pockets)
    rewrite_out_file(pdb_out_file, pockets)

In [72]:
read_druggability(fpocket_out_folder+'/6v5b_info.txt')

{0: '0.031',
 1: '0.003',
 2: '0.002',
 3: '0.004',
 4: '0.017',
 5: '0.036',
 6: '0.109',
 7: '0.000',
 8: '0.108',
 9: '0.000',
 10: '0.001',
 11: '0.002',
 12: '0.085',
 13: '0.001',
 14: '0.001',
 15: '0.000',
 16: '0.003',
 17: '0.002',
 18: '0.528',
 19: '0.005',
 20: '0.011',
 21: '0.000',
 22: '0.011',
 23: '0.007',
 24: '0.023',
 25: '0.001',
 26: '0.000',
 27: '0.000',
 28: '0.001',
 29: '0.002',
 30: '0.001',
 31: '0.001',
 32: '0.020',
 33: '0.000',
 34: '0.001',
 35: '0.000',
 36: '0.001',
 37: '0.000',
 38: '0.003',
 39: '0.001',
 40: '0.081',
 41: '0.001',
 42: '0.003',
 43: '0.001',
 44: '0.000',
 45: '0.001',
 46: '0.000',
 47: '0.001',
 48: '0.001',
 49: '0.003',
 50: '0.000',
 51: '0.001',
 52: '0.001',
 53: '0.000',
 54: '0.000',
 55: '0.002',
 56: '0.003',
 57: '0.888',
 58: '0.028',
 59: '0.000',
 60: '0.000',
 61: '0.001',
 62: '0.012',
 63: '0.001',
 64: '0.000',
 65: '0.002',
 66: '0.003',
 67: '0.000',
 68: '0.001',
 69: '0.002',
 70: '0.006',
 71: '0.226',
 7

# 输出文件

In [37]:
19, 58